In [ ]:
using Pkg
using CSV
using DataFrames
using Flux
using Flux: onehot, onehotbatch, logitcrossentropy, reset!, throttle
using Random
using Statistics: mean
Pkg.add("Parameters")
using Parameters: @with_kw

In [3]:
function build_model(args)
    scanner = Chain(Dense(args.word_list_len, args.N, σ), LSTM(args.N, args.N))
    #returns vector of length N of hidden states
    encoder = Dense(args.N, 1, identity)
    #encoder takes last element of scanner
        #take output and sum all (bc continuous variable) so that args.langs_len = 1
    #only want last hidden state 
    return scanner, encoder
end

build_model (generic function with 1 method)

In [4]:
function model(x, scanner, encoder)
    state = scanner.(x.data)[end]
    reset!(scanner) #reset hidden states so each sentence gets own treatment
    encoder(state)
end

model (generic function with 1 method)

In [5]:
function train()
    # Load Data
    train_data, test_data, arg = process_data()
    epochs = 50  
    epoch_num = []
    loss_values = []
  
    @info("Constructing Model...")
    scanner, encoder = build_model(arg)

    loss(x, y) = Flux.mse(model(x, scanner, encoder), y)
    #logitcrossentropy for discrete (classification), MSE for continuous (regression/predictive)
    testloss() = mean(loss(t...) for t in test_data)
    
    opt = ADAM(arg.lr)
    ps = params(scanner, encoder)
    #telling it what you want to train, could be just scanner but here           #we want both
    
    print(string(epochs, " *** epochs")) 
    @info("Training...")
    #train more than once, put in loop
    for i = 1:epochs
        e =  testloss()
        evalcb =  () -> @show e
        push!(loss_values, e)
        push!(epoch_num, i)
        Flux.train!(loss, ps, train_data, opt, cb = throttle(evalcb,                 arg.throttle))
    end
    
    df = DataFrame()
    df.EpochNum = epoch_num
    df.LossVals = loss_values

    CSV.write("test_loss.csv", df)
end

train (generic function with 1 method)

In [ ]:
train()